In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import PorterStemmer,WordNetLemmatizer
# from sklearn.linear_model import LogisticRegression,SGDClassifier
# from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [0]:
  from google.colab import drive 
  drive.mount("/content/gdrive")

In [0]:
!ls "/content/gdrive/My Drive/Datasets"

In [0]:
true = pd.read_csv("/content/gdrive/My Drive/Datasets/True.csv")
false = pd.read_csv("/content/gdrive/My Drive/Datasets/Fake.csv")

In [0]:
true.shape

In [0]:
true.head()

In [0]:
true['category'] = 1
false['category'] = 0

In [0]:
df = pd.concat([true,false]) #Merging the 2 datasets

In [0]:
df.head()

In [0]:
df.isna().sum() # Checking for nan Values

In [0]:
df.title.count()

In [0]:
df.subject.value_counts()

In [0]:
df['text'] = df['text'] + " " + df['title'] + " " + df['subject']
del df['title']
del df['subject']
del df['date']

In [0]:
df.shape


In [0]:
##for test purpose only
# df=df[:10000]

In [0]:
nltk.download('stopwords')
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [0]:
stemmer = PorterStemmer()
def stem_text(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            word = stemmer.stem(i.strip())
            final_text.append(word)
    return " ".join(final_text)        

In [0]:
df.text = df.text.apply(stem_text)

In [0]:
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 300 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(" ".join(df.text))
plt.imshow(wc , interpolation = 'bilinear')

In [0]:
x_train,x_test,y_train,y_test = train_test_split(df.text,df.category)

In [0]:
# cv=CountVectorizer(min_df=0,max_df=1,ngram_range=(1,2))
cv=CountVectorizer()

#transformed train reviews
cv_train_reviews=cv.fit_transform(x_train)
#transformed test reviews
cv_test_reviews=cv.transform(x_test)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

In [0]:
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = cv_train_reviews.shape[1]))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(Dense(units = 25 , activation = 'relu'))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))

In [0]:
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [0]:
model.fit(cv_train_reviews[:],y_train[:] , epochs = 5)

In [0]:
pred = model.predict(cv_test_reviews)

In [0]:
eg=["corona virus wiped the world "]
eg=cv.transform(eg)
model.predict(eg)

In [0]:
for i in range(len(pred)):
    if(pred[i] > 0.5):
        pred[i] = 1
    else:
        pred[i] = 0

In [0]:
accuracy_score(pred,y_test)

In [0]:
cv_report = classification_report(y_test,pred,target_names = ['0','1'])
print(cv_report)

In [0]:
cm_cv = confusion_matrix(y_test,pred)
cm_cv

In [0]:
cm_cv = pd.DataFrame(cm_cv, index=[0,1], columns=[0,1])
cm_cv.index.name = 'Actual'
cm_cv.columns.name = 'Predicted'

In [0]:
plt.figure(figsize = (10,10))
sns.heatmap(cm_cv,cmap= "Blues",annot = True, fmt='')

In [0]:
import pickle 
  
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(model) 